# Exploring ODF Extraction - BP Data Analysis
## Sam Potter
## Current: 5/16/19

## Path and Imports

In [ ]:
import sys
import os

# psfdi
sys.path.extend(['C:\\Users\\potterst1\\Desktop\Repositories\Github\psfdi',
                 'C:/Users/potterst1/Desktop/Repositories/Github/psfdi'])
sys.path.extend(['/workspace/stpotter/git/github/psfdi'])

from psfdi import visualize
from psfdi import fileIO
from matplotlib import pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import patches
from matplotlib import rc
from psfdi import utilities
from psfdi import odf
from psfdi import image_processing as imp
import seaborn as sns
import cv2
import matplotlib

from ipywidgets import *
from scipy import optimize as sciopt
from scipy.integrate import trapz
from scipy.interpolate import interp1d

## Magics

In [ ]:
%matplotlib inline

# Visualize Axis Confirmation Images

In [ ]:
paper_data_path = 'C:\\Users\\potterst1\\Box Sync\\Research\\Projects\\IGA DIC pSFDI\\Experimental Data\\2D pSFDI data\\ODF Extraction\\4.28.19\\3) Paper\\Sample\\Vertical\\Green'
paper_data_path = '/workspace/stpotter/Data/pSFDI/2D-Biax-Data/4.28.19/3) Paper/Sample/Vertical/Green'

paper_pic = cv2.imread(os.path.join(paper_data_path, 'planar_angle_0.tiff'), -1)

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.imshow(paper_pic, cmap='gray');

# BP Data

## Load Raw Intensity pSFDI Data - 360 Degrees

### Specify file paths

In [ ]:
bp_raw_path = 'C:\\Users\\potterst1\\Box Sync\\Research\\Projects\\IGA DIC pSFDI\\Experimental Data\\2D pSFDI data\\ODF Extraction\\4.28.19\\2) pSFDI\\Sample\\Vertical\\Green'
bp_raw_path = '/workspace/stpotter/Data/pSFDI/2D-Biax-Data/4.28.19/2) pSFDI/Sample/Vertical/Green'

bp_standard_path = 'C:\\Users\\potterst1\\Box Sync\\Research\Projects\\IGA DIC pSFDI\\Experimental Data\\2D pSFDI data\\ODF Extraction\\4.28.19\\1) Standard\\Standard\\Vertical\\Green'
bp_standard_path = '/workspace/stpotter/Data/pSFDI/2D-Biax-Data/4.28.19/1) Standard/Sample/Vertical/Green'

### Specify spatial frequency and polarizer resolution

In [ ]:
sfx_per = 36.4679
polar_res = 4
polar_max = 360
polar_angles = np.arange(0, polar_max, polar_res)

bp_planar = [cv2.imread(os.path.join(bp_raw_path, 'planar_angle_' + str(angle) + '.tiff'), -1)for angle in polar_angles]
bp_planar = np.array(bp_planar)

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.imshow(bp_planar[0, :, :], cmap='gray');

## Crop pSFDI ROI - Doing this manually for now

In [ ]:
rowstart, rowstop = 175, 775  # Best: 175, 775
colstart, colstop = 350, 925  # Best: 350, 925

bp_planar = bp_planar[:, rowstart:rowstop, colstart:colstop]

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.imshow(bp_planar[0, :, :], cmap='gray');

## Load in SALS data

In [ ]:
SALS_data_path = 'C:\\Users\\potterst1\\Box Sync\\Research\\Projects\\IGA DIC pSFDI\\Experimental Data\\2D pSFDI data\\ODF Extraction\\SALS\\BP\\Results\\BP ODF 042919'
SALS_data_path = '/workspace/stpotter/Data/pSFDI/SALS/BP ODF 042919'

#SALS_data_dict = fileIO.read_SALS(os.path.join(SALS_data_path, 'SALSA OUT TXT\\BP_ODF_42919_SALSA.txt'))
SALS_data_dict = fileIO.read_SALS(os.path.join(SALS_data_path, 'SALSA OUT TXT/BP_ODF_42919_SALSA.txt'))

In [ ]:
sals_pd_im = cv2.imread(os.path.join(SALS_data_path, 'Images\\BP_ODF_42919_PrefD.png'), -1)
sals_pd_im = cv2.imread(os.path.join(SALS_data_path, 'Images/BP_ODF_42919_PrefD.png'), -1)
sals_pd_im = cv2.cvtColor(sals_pd_im, cv2.COLOR_BGR2RGB)

In [ ]:
fig = plt.figure(figsize=(15, 15))
plt.title('SALSA Processed PD')
plt.imshow(sals_pd_im);

In [ ]:
x = SALS_data_dict['x']
y = SALS_data_dict['y']

PD_2d = SALS_data_dict['PD']
SD_2d = SALS_data_dict['SD']

U = np.cos(np.deg2rad(PD_2d))
V = np.sin(np.deg2rad(PD_2d))

In [ ]:
# SALS PD
fig = plt.figure(figsize=(15, 15))
im0 = plt.imshow(PD_2d, cmap='hsv')
plt.quiver(U, V, headlength=0, headaxislength=0);
plt.title('SALS PD')
ax = plt.gca()
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
colorlimits = (0, 180);
im0.set_clim(colorlimits)
plt.colorbar(im0, cax=cax);

In [ ]:
# SALS SD
fig = plt.figure(figsize=(15, 15))
im0 = plt.imshow(SD_2d, cmap='jet')
plt.title('SALS SD')
ax = plt.gca()
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
colorlimits = (45, 55);
im0.set_clim(colorlimits)
plt.colorbar(im0, cax=cax);

## Register Images

### Create pSFDI Binary Mask

In [ ]:
bp_planar_reg = bp_planar[0, :, :]
fig = plt.figure(figsize=(10, 10))
plt.imshow(bp_planar_reg, cmap='gray')
plt.imsave(os.path.join(bp_raw_path, 'cropped_for_mask.png'), bp_planar_reg, cmap='gray')

### View Binary Masks before upscaling

In [ ]:
psfdi_mask = cv2.imread(os.path.join(bp_raw_path, 'Mask.tif'), -1)
#sals_mask = cv2.imread(os.path.join(SALS_data_path, 'BW_Images\\BP_ODF_42919.png'), -1)
sals_mask = cv2.imread(os.path.join(SALS_data_path, 'BW_Images/BP_ODF_42919.png'), -1)
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 10))
im0 = ax0.imshow(psfdi_mask, cmap='gray');
ax0.set_title('pSFDI Mask for registration');

im1 = ax1.imshow(sals_mask, cmap='gray')
ax1.set_title('SALS mask for registration');

### Upscale Binary Masks

In [ ]:
psfdi_mask_upscale = imp.upscale(psfdi_mask, *PD_2d.shape)
sals_mask_upscale = imp.upscale(sals_mask[:, :, 0], *PD_2d.shape)

fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(15, 15))
im0 = ax0.imshow(psfdi_mask_upscale, cmap='gray');
ax0.set_title('pSFDI Mask for registration - upscaled');

im1 = ax1.imshow(sals_mask_upscale, cmap='gray')
ax1.set_title('SALS mask for registration- upscaled');

### Register

In [ ]:
# Register images
psfdi_mask_registered, warp_data = imp.register(psfdi_mask_upscale.astype(np.uint8), sals_mask_upscale.astype(np.uint8))

print('cc: {}'.format(warp_data['cc']))
print('warp matrix:')
print(warp_data['warp'])

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(15, 15))
im0 = ax0.imshow(psfdi_mask_registered, cmap='gray');
ax0.set_title('pSFDI Mask for registration - registered');

im1 = ax1.imshow(sals_mask_upscale, cmap='gray')
ax1.set_title('SALS mask for registration- registered');

### Upscale and Register all the Planar pSFDI Data

In [ ]:
# Upscale
bp_planar_upscale = [imp.upscale(bp_planar[angle, :, :], *PD_2d.shape) for angle in range(len(polar_angles))]
bp_planar_upscale = np.array(bp_planar_upscale)

In [ ]:
# Register
bp_planar_registered = [imp.warp(bp_planar_upscale[angle, :, :], warp_data['warp']) for angle in range(len(polar_angles))]
bp_planar_registered = np.array(bp_planar_upscale)

## Feasibility Study - Raw Data

### Set the row and column data for visualization etc

In [ ]:
row = 15
col = 15
ydim = SALS_data_dict['PD'].shape[0]

index = col * ydim + (ydim - col)

### Visualize where the point is

In [ ]:
row_step = int(bp_planar.shape[1] / PD_2d.shape[0])
col_step = int(bp_planar.shape[2] / PD_2d.shape[1])

# Get the correct rectangle start indices
psfdi_row = row * row_step
psfdi_col = col * col_step

fig, ax = plt.subplots(figsize=(10, 10))
plt.imshow(bp_planar[0, :, :], cmap='gray');
rect = patches.Rectangle((psfdi_col, psfdi_row), row_step, col_step, edgecolor='r', facecolor='r')
ax.add_patch(rect);
plt.title('Location of Beam ROI - Shown in Red');

### Get SALS data

In [ ]:
odf_theta = SALS_data_dict['theta'][index, :]
odf_gamma = SALS_data_dict['odf'][index, :]

### Get pSFDI data with roll compensation

In [ ]:
roll = 90 // polar_res

bp_data = bp_planar_registered[:, row, col]
bp_data = np.roll(bp_data, roll)

### Convert polar angles to radians for analysis

In [ ]:
polar_angles = np.deg2rad(polar_angles)

### Visualize Data

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(15, 15), subplot_kw=dict(projection='polar'))
ax0.plot(polar_angles, bp_data, linestyle='--', marker='o', color='g',
         label='Mean Fiber Intensity');
ax0.set_title('Upscaled pSFDI Raw Intensitym - Roll Compensated');
ax0.set_ylim([0.975 * np.min(bp_data), 1.025 * np.max(bp_data)])

ax1.plot(odf_theta, odf_gamma, color='r', label='SALS ODF');
ax1.set_ylim([0, 0.25])
ax1.set_title('SALS ODF');

In [ ]:
fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(15, 15))
ax0.plot(polar_angles, bp_data, linestyle='--', marker='o', color='g',
         label='Mean Fiber Intensity');
ax0.set_title('Upscaled pSFDI Raw Intensitym - Roll Compensated');
ax0.set_ylim([np.min(bp_data), np.max(bp_data)])
ax0.set_xlabel('Theta')

ax1.plot(odf_theta, odf_gamma, color='r', label='SALS ODF');
ax1.set_ylim([np.min(odf_gamma), np.max(odf_gamma)])
ax1.set_title('SALS ODF');
ax1.set_xlabel('Theta');

## Fourier Series Fit

### SALS Data

In [ ]:
odf_theta = SALS_data_dict['theta'][index, :]
odf_gamma = SALS_data_dict['odf'][index, :]

In [ ]:
an, bn, c = odf.fit_fourier(20, odf_gamma, odf_theta)
odf_fsfit = odf.compute_fourier(an, bn, c, odf_theta)

In [ ]:
fig = plt.figure(figsize=(15, 10))
plt.scatter(np.rad2deg(odf_theta), odf_gamma, marker='o', s=10, label='SALS ODF Data')
plt.plot(np.rad2deg(odf_theta), odf_fsfit, color='r', label='Fourier Series Fit');
plt.ylabel('Gamma');
plt.xlabel('Theta');
plt.legend()
plt.title('SALS ODF Data and Associated Fourer Series Fit');

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.polar(odf_theta, odf_fsfit, color='r', label='Fourier Series Fit');
plt.scatter(odf_theta, odf_gamma, s=10, label='SALS ODF Data');
plt.legend()
plt.title('SALS ODF Data and Associated Fourier Series Fit - Polar Plot');

## Compute the Primary and Secondary Directions

In [ ]:
theta1, theta2 = odf.structural_eigenval_thetas(an[0], bn[0])

In [ ]:
print('Theta I (deg): {}'.format(theta1))
print('Theta II (deg): {}'.format(theta2))

In [ ]:
fig = plt.figure(figsize=(15, 10))
plt.scatter(np.rad2deg(odf_theta), odf_gamma, s=10, label='SALS ODF Data');
plt.plot(np.rad2deg(odf_theta), odf_fsfit, color='r', label='Fourier Series Fit')
plt.plot((theta1 + 180) * np.ones(len(odf_theta)), np.linspace(0.12, 0.20, len(odf_theta)), color='k', label='Theta I')
plt.plot(theta2 * np.ones(len(odf_theta)), np.linspace(0.12, 0.20, len(odf_theta)), color='g', label='Theta II')
plt.plot((theta2 + 180) * np.ones(len(odf_theta)), np.linspace(0.12, 0.20, len(odf_theta)), color='g', label='Theta II')
plt.xlabel('Theta')
plt.ylabel('Gamma')
plt.legend();
plt.title('SALS ODF Data and Associated Fourier Series Fit - Primary Directions from Structural Tensor');

In [ ]:
theta1_r = np.deg2rad(theta1)
theta2_r = np.deg2rad(theta2)
fig = plt.figure(figsize=(10, 10))
plt.polar(odf_theta, odf_fsfit, color='r', label='Fourier Series Fit');
plt.scatter(odf_theta, odf_gamma, s=10, label='SALS ODF Data');
plt.polar([theta1_r, theta1_r + np.pi], [1.01 * np.max(odf_gamma), 1.01 * np.max(odf_gamma)], color='k', label='Theta I')
plt.polar([theta2_r, theta2_r + np.pi], [1.01 * np.max(odf_gamma), 1.01 * np.max(odf_gamma)], color='g', label='Theta II')
plt.legend()
plt.title('SALS Data and Fourier Series Fit - Primary Directions on Polar Plot');

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(15, 15), subplot_kw=dict(projection='polar'))
ax0.plot(polar_angles, bp_data, linestyle='--', marker='o', color='g',
         label='Mean Fiber Intensity');
ax0.set_title('Upscaled pSFDI Raw Intensity - Roll Compensated');
ax0.set_ylim([0.975 * np.min(bp_data), 1.025 * np.max(bp_data)])
ax0.plot([theta1_r, theta1_r + np.pi], [1.01 * np.max(bp_data), 1.01 * np.max(bp_data)], color='k', label='Theta I')
ax0.plot([theta2_r, theta2_r + np.pi], [1.01 * np.max(bp_data), 1.01 * np.max(bp_data)], color='g', label='Theta II')

ax1.plot(odf_theta, odf_gamma, color='r', label='SALS ODF');
ax1.plot([theta1_r, theta1_r + np.pi], [1.01 * np.max(odf_gamma), 1.01 * np.max(odf_gamma)], color='k', label='Theta I')
ax1.plot([theta2_r, theta2_r + np.pi], [1.01 * np.max(odf_gamma), 1.01 * np.max(odf_gamma)], color='g', label='Theta II')
ax1.set_ylim([0, 0.25])
ax1.set_title('SALS ODF');
plt.legend();

## Fit pSFDI Ellipse

In [ ]:
theta = np.linspace(0, 2 * np.pi, 100)
epts = odf.ellipse(33750, .5, 0, theta)
fig = plt.figure(figsize=(10, 10))
plt.polar(theta, epts, label='Bogus Ellipse Plot')
plt.polar(polar_angles, bp_data, linestyle='--', marker='o', color='g', label='pSFDI Intensity Data')
ax0.set_ylim([0.9 * np.min(epts), 1.1 * np.max(epts)])
plt.legend();

In [ ]:
args_tup = (polar_angles, bp_data)
int_x = [35000, 0, 0]
bounds = ((0, 1000 * np.max(bp_data)), (0.0, 1.0), (0, 2 * np.pi))

### Global

In [ ]:
minfun = odf.minellipsefun
res = sciopt.differential_evolution(minfun, args=args_tup, bounds=bounds)
print('Final values of b, e, phi: {}'.format(res.x))
print('Final residual: {}'.format(res.fun))
print('Final optimizer success status: {}'.format(res.success))
print('Final optimizer message: {}'.format(res.message))

In [ ]:
fit_ellipse = odf.ellipse(res.x[0], res.x[1], res.x[2], polar_angles)
fig = plt.figure(figsize=(10, 10))
plt.polar(polar_angles, fit_ellipse, marker='o')
plt.polar(polar_angles, bp_data, marker='o', color='g')
plt.ylim([33500, 34000]);

## Remove Pi-Span

### SALS data

In [ ]:
lower = odf_theta > np.deg2rad(theta2) 
upper = odf_theta < np.deg2rad(theta2 + 180)
indices = np.all((lower, upper), axis=0)

In [ ]:
theta_pi_span = odf_theta[indices]
# Shift this to 0 deg
theta_pi_span = theta_pi_span - np.min(theta_pi_span) * np.ones(len(theta_pi_span))
# Shift this to -pi/2
theta_pi_span = theta_pi_span - np.pi / 2 * np.ones(len(theta_pi_span))
gamma_pi_span = odf_gamma[indices]

In [ ]:
# gamma_pi_span = gamma_pi_span - np.min(gamma_pi_span) * np.ones(len(gamma_pi_span))

In [ ]:
fig = plt.figure(figsize=(15, 10))
plt.plot(theta_pi_span, gamma_pi_span, color='r');
plt.title('Pi-Span of SALS ODF');
plt.xlabel('Theta');

In [ ]:
gamma_area = trapz(gamma_pi_span, theta_pi_span)

norm_gamma_pi_span = gamma_pi_span / gamma_area

print('Unit Normalized ODF Data Area: {}'.format(trapz(norm_gamma_pi_span, theta_pi_span)))

In [ ]:
fig = plt.figure(figsize=(15, 10))
plt.plot(np.rad2deg(theta_pi_span), norm_gamma_pi_span, color='r');
plt.title('Pi-Span of SALS ODF - Unit Normalized');
plt.xlabel('Theta');

### pSFDI Intensity

In [ ]:
lower = polar_angles > np.deg2rad(theta2) 
upper = polar_angles < np.deg2rad(theta2 + 180)
indices = np.all((lower, upper), axis=0)

In [ ]:
psfdi_theta_pi_span = polar_angles[indices]
# Shift this to 0 deg
psfdi_theta_pi_span = psfdi_theta_pi_span - np.min(psfdi_theta_pi_span) * np.ones(len(psfdi_theta_pi_span))
# Shift this to -pi/2
psfdi_theta_pi_span = psfdi_theta_pi_span - np.pi / 2 * np.ones(len(psfdi_theta_pi_span))
intensity_pi_span = bp_data[indices]

In [ ]:
fig = plt.figure(figsize=(15, 10))
plt.plot(psfdi_theta_pi_span, intensity_pi_span, linestyle='--', marker='o', color='g');
plt.title('Pi-Span of pSFDI Intensity');
plt.xlabel('Theta');

In [ ]:
intensity_area = trapz(intensity_pi_span, psfdi_theta_pi_span)

norm_intensity_pi_span = intensity_pi_span / intensity_area

print('Unit Normalized Intensity Data Area: {}'.format(trapz(norm_intensity_pi_span, psfdi_theta_pi_span)))

In [ ]:
fig = plt.figure(figsize=(15, 10))
plt.plot(psfdi_theta_pi_span, norm_intensity_pi_span, linestyle='--', marker='o', color='g');
plt.title('Pi-Span of pSFDI Intensity - Unit Normalized');
plt.xlabel('Theta');

### Compute Mean and SD of SALS Pi-Span

In [ ]:
gamma_mean = trapz(theta_pi_span * gamma_pi_span, theta_pi_span)
gamma_var = trapz(gamma_pi_span * np.square(theta_pi_span - np.ones(len(theta_pi_span)) * gamma_mean), theta_pi_span)
gamma_sd = np.sqrt(gamma_var)

In [ ]:
print('Mean of Pi-Span (deg): {}'.format(np.rad2deg(gamma_mean)))
print('Standard Deviation of Pi-Span (deg): {}'.format(np.rad2deg(gamma_sd)))

### Compute beta distribution

In [ ]:
# Compute the beta values
sals_beta = odf.compute_beta(gamma_mean, gamma_sd, theta_pi_span)

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.plot(theta_pi_span,sals_beta, color='k', label='Beta Distribution');
plt.plot(theta_pi_span, norm_gamma_pi_span, color='r', label='SALS ODF')
plt.title('SALS ODF and Beta Distribution');
plt.xlabel('Theta')
plt.legend();

## Wrapped Normal Distribution

In [ ]:
test_wn = odf.wrapped_normal(gamma_mean, gamma_sd, np.pi, theta_pi_span)
fig = plt.figure(figsize=(10, 10))
plt.plot(theta_pi_span, test_wn);

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.plot(theta_pi_span, test_wn, color='k', label='Wrapped Normal Distribution');
plt.plot(theta_pi_span, norm_gamma_pi_span, color='r', label='SALS ODF')
plt.title('SALS ODF and Beta Distribution');
plt.xlabel('Theta')
plt.legend();

# Extracting Single Fiber Values Using Discrete Sampling

## Use wrapped normal distribution

### Generate Wrapped Normal Samples

In [ ]:
test_wn = odf.wrapped_normal(gamma_mean, gamma_sd, np.pi, theta_pi_span)
fig = plt.figure(figsize=(10, 10))
plt.plot(theta_pi_span, test_wn);
plt.title('Wrapped Normal Distribution with Mean and Std. Dev from SALS Data');

In [ ]:
test_samples = odf.sample_wrappednormal(gamma_mean, gamma_sd, np.pi, 100000)
fig = plt.figure(figsize=(10, 10))
N, bins, patches = plt.hist(test_samples, density=True)
plt.plot(theta_pi_span, test_wn);
plt.title('Sampled Wrapped Normal vs SALS measured Wrapped Normal');
plt.xlabel('Theta');

## Test out the Discrete Convolution

### Single Fiber

In [ ]:
a0test, a2test, a4test, = 0.3, 0.01, 0.01
radThetas = np.linspace(-np.pi / 2, np.pi / 2, 100)
testI = odf.IfiberRad(a0test, a2test, a4test, gamma_mean, radThetas)

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.plot(radThetas, testI)
plt.title('Test I Single Fiber');

### Fiber Distributions

In [ ]:
testIdist = odf.IdistDiscrete(a0test, a2test, a4test, test_samples, radThetas)
fig = plt.figure(figsize=(10, 10))
plt.plot(radThetas, testI, label='Single Fiber')
plt.plot(radThetas, testIdist, label='Fiber Distribution')
plt.legend();

### Test minimization

In [ ]:
args = (test_samples, radThetas, testIdist)
int_x = np.array([0, 0, 0])
bounds = ((0, 1), (-1, 1), (0, 1))
minfun = odf.minfunDiscrete
res = sciopt.minimize(minfun, int_x, args=args, bounds=bounds, method='L-BFGS-B')
print('Final values of a0, a2, a4: {}'.format(res.x))
print('Synthetic values of a0, a2, a4: {}'.format(np.array([a0test, a2test, a4test])))
print('Final residual: {}'.format(res.fun))
print('Final optimizer success status: {}'.format(res.success))
print('Final optimizer message: {}'.format(res.message))

### Plot Test Minimization Results

In [ ]:
fitFiber = odf.IfiberRad(res.x[0], res.x[1], res.x[2], gamma_mean, radThetas)
fig = plt.figure(figsize=(10, 10))
plt.plot(radThetas, testI, label='Synthetic Single Fiber')
plt.plot(radThetas, fitFiber, marker='o', label='Fit Single Fiber')
plt.legend();

## Fit pSFDI Data

### Prep for minimizing

In [ ]:
args = (test_samples, psfdi_theta_pi_span, norm_intensity_pi_span)
int_x = np.array([0, 0, 0])  # Good enough to guess all zeros
bounds = ((0, 1), (-1, 1), (0, 1))

### Minimize and report results

In [ ]:
minfun = odf.minfunDiscrete
res = sciopt.minimize(minfun, int_x, args=args, bounds=bounds, method='L-BFGS-B')
print('Final values of a0, a2, a4: {}'.format(res.x))
print('Final residual: {}'.format(res.fun))
print('Final optimizer success status: {}'.format(res.success))
print('Final optimizer message: {}'.format(res.message))

### Plot resulting fit

In [ ]:
a0_fit_wn = res.x[0]
a2_fit_wn = res.x[1]
a4_fit_wn = res.x[2]

fit_Idist_wn = odf.IdistDiscrete(a0_fit_wn, a2_fit_wn, a4_fit_wn, test_samples, psfdi_theta_pi_span)

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.scatter(psfdi_theta_pi_span, norm_intensity_pi_span, color='g', label='Raw Intensity Data')
plt.plot(psfdi_theta_pi_span, fit_Idist_wn, color='k', label='Distribution from Fit Single Fiber Terms');
plt.ylim([0.999 * np.min(norm_intensity_pi_span), 1.001 * np.max(norm_intensity_pi_span)]);
plt.title('Single Fiber ai terms run through wn vs. Raw Intensity Data');
plt.xlabel('Theta');
plt.legend();

# Extracting Distribution Statistics

## Test out Discrete Convolution

### Single Fiber

In [ ]:
a0test, a2test, a4test, = 0.3, 0.01, 0.01
radThetas = np.linspace(-np.pi / 2, np.pi / 2, 100)
testI = odf.IfiberRad(a0test, a2test, a4test, gamma_mean, radThetas)
fig = plt.figure(figsize=(10, 10))
plt.plot(radThetas, testI)
plt.title('Test I Single Fiber');

### Fiber Distributions

In [ ]:
testIdist = odf.IdistDiscrete(a0test, a2test, a4test, test_samples, radThetas)
fig = plt.figure(figsize=(10, 10))
plt.plot(radThetas, testI, label='Single Fiber')
plt.plot(radThetas, testIdist, label='Fiber Distribution')
plt.legend();

### Test minimization

In [ ]:
args = (test_samples, radThetas, testIdist)
int_x = np.array([0, 0, 0])
bounds = ((0, 1), (-1, 1), (0, 1))
minfun = odf.minfunDiscrete
res = sciopt.minimize(minfun, int_x, args=args, bounds=bounds, method='L-BFGS-B')
print('Final values of a0, a2, a4: {}'.format(res.x))
print('Synthetic values of a0, a2, a4: {}'.format(np.array([a0test, a2test, a4test])))
print('Final residual: {}'.format(res.fun))
print('Final optimizer success status: {}'.format(res.success))
print('Final optimizer message: {}'.format(res.message))